<a href="https://colab.research.google.com/github/JorgeZarama/Supervisado/blob/main/Programa11_EvaluacionClasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_predict, cross_validate, cross_val_score
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from pprint import pprint
from sklearn.svm import SVC
import pandas as pd

In [53]:
# Carga de datos.
csv = pd.read_csv('Dataset_spine_2C.csv', sep=',')
print(csv.head())
# Convertir dataframe en numpy array
data = np.array(csv)
X = data[:,:-2]
y = data[:,-2]

ocur, frec = np.unique(y, return_counts=True)
print(ocur, frec)

        Col1       Col2       Col3       Col4        Col5       Col6  \
0  63.027817  22.552586  39.609117  40.475232   98.672917  -0.254400   
1  39.056951  10.060991  25.015378  28.995960  114.405425   4.564259   
2  68.832021  22.218482  50.092194  46.613539  105.985135  -3.530317   
3  69.297008  24.652878  44.311238  44.644130  101.868495  11.211523   
4  49.712859   9.652075  28.317406  40.060784  108.168725   7.918501   

       Col7     Col8     Col9     Col10      Col11    Col12 Class_att  \
0  0.744503  12.5661  14.5386  15.30468 -28.658501  43.5123  Abnormal   
1  0.415186  12.8874  17.5323  16.78486 -25.530607  16.1102  Abnormal   
2  0.474889  26.8343  17.4861  16.65897 -29.031888  19.2221  Abnormal   
3  0.369345  23.5603  12.7074  11.42447 -30.470246  18.8329  Abnormal   
4  0.543360  35.4940  15.9546   8.87237 -16.378376  24.9171  Abnormal   

                                         Unnamed: 13  
0                                                NaN  
1                 

In [54]:
# Partición EXTERNA. Test: hold-out split 80-20%.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Train dimensions: ', np.shape(X_train))
print('Test dimensions:  ', np.shape(X_test))


Train dimensions:  (248, 12)
Test dimensions:   (62, 12)


In [55]:
# Estandarización de los datos de entrenamiento y test
standardizer = preprocessing.StandardScaler()
X_train = standardizer.fit_transform(X_train)
X_test = standardizer.transform(X_test)

In [61]:
# Definimos el algoritmo
# C=0.1, gamma='auto' y kernel gausiano
alg = SVC(C=1, gamma='auto', kernel='rbf')

In [63]:
# Validación y obtención de las predicciones del modelo.
# en "cv=KFold(n_splits=5)" se hace el cross-validation INTERNO!!
# el comando "cross_val_predict" nos permite extraer directamente las predicciones de las 5 bolsas de validación
y_pred = cross_val_predict(alg, X_train, y_train, cv = KFold(n_splits=5, shuffle=True, random_state=42))
print(y_pred)

['Abnormal' 'Abnormal' 'Abnormal' 'Normal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Normal' 'Normal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Normal' 'Abnormal' 'Abnormal' 'Abnormal' 'Normal' 'Abnormal'
 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Normal' 'Normal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Normal' 'Abnormal' 'Normal' 'Normal' 'Abnormal' 'Normal'
 'Normal' 'Normal' 'Abnormal' 'Abnormal' 'Abnormal' 'Normal' 'Abnormal'
 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Normal'
 'Abnormal' 'Normal' 'Normal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Normal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Normal'
 'Abnormal' 'Abnormal' 'Normal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Normal' 'Normal' 'Normal' 'Abnormal' 'Normal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Normal' 'Abnormal'
 'Abnormal' 'Abnormal' '

In [65]:
# Presentación de los resultados de la evaluación.
print("Exactitud: %.3f" % (metrics.accuracy_score(y_train, y_pred))) # accuracy
print("Precisión: %.3f" % (metrics.precision_score(y_train, y_pred, labels=['Normal'], average="micro"))) # precision
print("Sensibilidad: %.3f" % (metrics.recall_score(y_train, y_pred, labels=['Normal'], average="micro"))) # sensibilidad
print("F1-score: %.3f" % (metrics.f1_score(y_train, y_pred, labels=['Normal'], average="micro"))) # F-score


Exactitud: 0.774
Precisión: 0.671
Sensibilidad: 0.622
F1-score: 0.646


In [66]:
# Extraemos la matriz de confusión
print("Matriz de confusión:\n", metrics.confusion_matrix(y_train, y_pred))

Matriz de confusión:
 [[141  25]
 [ 31  51]]


In [67]:
print("Tabla de métricas:\n", metrics.classification_report(y_train, y_pred))

Tabla de métricas:
               precision    recall  f1-score   support

    Abnormal       0.82      0.85      0.83       166
      Normal       0.67      0.62      0.65        82

    accuracy                           0.77       248
   macro avg       0.75      0.74      0.74       248
weighted avg       0.77      0.77      0.77       248



In [44]:
# Una vez hemos optimizado los hiperparámetros con el conjunto de validación, utilizamos todos los datos de
# "train" y "validation" para entrenar un modelo definitivo con esos hiperparámetros ajustados
model = SVC(C=10, gamma='scale', kernel='rbf') # Definimos el modelo con los hiperparámetros óptimos
model = model.fit(X_train, y_train) # Entrenamos el modelo con todos los datos de "train" + "val"

# y finalmente evaluamos el test con el método "score()"
test_results = model.score(X_test, y_test) # Evaluamos la exactitud del test
print('Exactitud en test: ', np.round(test_results*100,4), '%')

Exactitud en test:  87.0968 %


In [45]:
# obtenemos las predicciones del test para extraer el resto de métricas de interés
y_pred_test = model.predict(X_test) # Extraer predicciones
print(metrics.classification_report(y_test, y_pred_test)) # Aplicar el método de classification_report()
print(metrics.confusion_matrix(y_test, y_pred_test)) # Extraer la matriz de confusión

              precision    recall  f1-score   support

    Abnormal       0.93      0.89      0.91        44
      Normal       0.75      0.83      0.79        18

    accuracy                           0.87        62
   macro avg       0.84      0.86      0.85        62
weighted avg       0.88      0.87      0.87        62

[[39  5]
 [ 3 15]]
